In [3]:
! pip install requests pandas dotenv openpyxl pyap

In [ ]:
import os
import re
import requests
import json
import time
import pandas as pd
import pyap
from urllib.parse import urlparse
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("TWENTY_KEY")
BASE_URL = os.getenv("TWENTY_BASE_URL", "http://twenty.local:3003")
HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}",
}


class RateLimitException(Exception):
    """Exception raised when hitting rate limits."""
    pass


def load_and_filter_data(filepath: str) -> pd.DataFrame:
    """
    Load the Excel file and remove any rows where 'Contacted?' or
    'Dogs / Cats?' are non-empty.
    """
    df = pd.read_excel(filepath, engine="openpyxl")

    # Remove duplicate charities based on their unique Charity Number
    df.drop_duplicates(subset=["Charity Number"], inplace=True)

    # Treat NaN as empty string for filtering
    df = df.fillna({"Contacted?": "", "Dogs / Cats?": ""})
    mask = (df["Contacted?"] == "") & (df["Dogs / Cats?"] == "")
    return df.loc[mask].copy()


def clean_domain(url: str) -> str:
    """
    Extract just the domain from a URL or hostname string,
    stripping any scheme (http/https), 'www.' prefix, and paths.
    """
    if not url or pd.isna(url):
        return ""
    # Ensure there's a scheme so urlparse will treat the input as a URL
    if not re.match(r"https?://", url):
        url = f"https://{url}"
    parsed = urlparse(url)
    # Strip 'www.' prefix if present (but only as a prefix)
    domain = parsed.netloc
    if domain.startswith("www."):
        domain = domain[4:]  # Remove the first 4 characters "www."
    return domain


def parse_income(value) -> int | None:
    """
    Convert a numeric or string income into micros (int).
    Returns None if the field is empty or invalid.
    """
    if pd.isna(value) or value == "":
        return None
    # Strip out non-digit/decimal characters
    if isinstance(value, str):
        numeric = re.sub(r"[^\d.]", "", value)
        if not numeric:
            return None
        amount = float(numeric)
    else:
        amount = float(value)
    return int(amount * 1_000_000)


def company_exists(domain: str) -> bool:
    """
    Query TwentyCRM for a company with the given domain using GraphQL.
    Returns True if at least one match is found.
    """
    query = """
    query GetCompanies($filter: CompanyFilterInput) {
        companies(filter: $filter) {
            edges {
                node {
                    id
                    domainName {
                        primaryLinkUrl
                    }
                }
            }
        }
    }
    """

    variables = {
        "filter": {
            "and": [
                {"domainName": {"primaryLinkUrl": {"ilike": f"%{domain}"}}},
                {
                    "or": [
                        {"deletedAt": {"is": "NULL"}},
                        {"deletedAt": {"is": "NOT_NULL"}},
                    ]
                },
            ]
        }
    }

    payload = {"query": query, "variables": variables}

    resp = requests.post(f"{BASE_URL}/graphql", headers=HEADERS, json=payload)
    resp.raise_for_status()
    data = resp.json()

    # Check for GraphQL errors
    if "errors" in data:
        # Check if any error contains "Too many requests"
        for error in data["errors"]:
            if "Too many requests" in error.get("message", ""):
                raise RateLimitException("Too many requests")
        # For other GraphQL errors, raise an exception instead of returning False
        raise Exception(f"GraphQL errors: {data['errors']}")

    # Check if any companies were found
    data_section = data.get("data")
    if not data_section:
        raise Exception("No 'data' section in response")

    companies_section = data_section.get("companies")
    if not companies_section:
        raise Exception("No 'companies' section in data")

    edges = companies_section.get("edges", [])

    return len(edges) > 0


def create_company(payload: dict) -> dict:
    """
    Create a new company in TwentyCRM with the given payload.
    """
    resp = requests.post(f"{BASE_URL}/rest/companies", headers=HEADERS, json=payload)
    resp.raise_for_status()
    return resp.json()


def extract_address_fields(raw_address: str) -> dict:
    """
    Parse a raw UK address string and return structured fields
    that map to TwentyCRM's address schema.
    """
    # Try to parse; country='GB' for United Kingdom
    addresses = pyap.parse(str(raw_address or ""), country="GB")

    if not addresses:
        # Fallback: empty fields + default country
        return {
            "addressStreet1": "",
            "addressCity": "",
            "addressState": "",
            "addressPostcode": "",
            "addressCountry": "United Kingdom",
        }

    # Take the first match
    addr_dict = addresses[0].as_dict()
    # Build street line: prefer full_street, else number + name
    street = addr_dict.get("full_street") or " ".join(
        filter(None, [addr_dict.get("street_number"), addr_dict.get("street_name")])
    )
    return {
        # "addressStreet1": street.strip(),
        "addressCity": (addr_dict.get("city") or "").strip(),
        "addressState": (addr_dict.get("region1") or "").strip(),
        "addressPostcode": (addr_dict.get("postal_code") or "").strip(),
        "addressCountry": "United Kingdom",
    }


def build_payload(row: pd.Series) -> dict:
    """
    Construct the JSON payload for a single row in the DataFrame,
    using pyap to extract UK address components.
    """
    domain = clean_domain(row["Website"])

    postCode = ""
    charity_postcode = row.get("Charity Postcode", "")
    if (
        charity_postcode
        and not pd.isna(charity_postcode)
        and str(charity_postcode).strip()
    ):
        postCode = str(charity_postcode).strip()
    # Extract structured address fields
    address_fields = extract_address_fields(
        f"{row.get("Charity Address", "")}, {postCode}"
    )

    payload = {
        "name": row["Charity Name"].title(),
        "charityNumber": str(row["Charity Number"]),
        "domainName": {
            "primaryLinkLabel": "",
            "primaryLinkUrl": domain,
            "additionalLinks": [],
        },
        "address": address_fields,
        "whatTheyDo": (
            row.get("Activities", "") or row.get("What the charity does", "")
        ),
    }

    # Add ARR if available
    micros = parse_income(row.get("Last Recorded Income", ""))
    if micros is not None:
        payload["annualRecurringRevenue"] = {
            "amountMicros": micros,
            "currencyCode": "GBP",
        }

    return payload


filepath = (
    "./full_uk_charitydetails_2025_07_08_11_28_38-"
    "more-than-200000-exp-and-hospital sanctuary rescue rehab "
    "rehabilitation wildlife marine.xlsx"
)
df = load_and_filter_data(filepath)

# Convert DataFrame to list of dictionaries to avoid iterator corruption
rows_to_process = df.to_dict("records")

# Initialize counters for summary
total_rows = len(rows_to_process)
created_count = 0
skipped_no_domain = 0
skipped_exists = 0
error_count = 0

for row_dict in rows_to_process:
    domain = clean_domain(row_dict["Website"])
    # print(f"Processing {row_dict['Charity Name']} with domain: {domain}")
    if not domain:
        # print(f"Skipping '{row_dict['Charity Name']}' (no valid domain)")
        skipped_no_domain += 1
        continue

    try:
        if company_exists(domain):
            # print(f"Already exists: {domain}")
            skipped_exists += 1
            continue

        payload = build_payload(pd.Series(row_dict))
        try:
            result = create_company(payload)
            print(
                f"Created: org for {domain}"
            )
            created_count += 1
        except requests.HTTPError as err:
            print(f"Error creating {domain}: {err}")
            error_count += 1

        
    except RateLimitException:
        print(f"Rate limit hit for {domain}, sleeping for 5 seconds...")
        time.sleep(5)
        # Retry the same iteration by not incrementing the loop
        # We need to decrement to retry this row
        continue
    except Exception as err:
        print(f"Error checking if company exists for {domain}: {err}")
        error_count += 1
        continue

# Summary
print("\n" + "="*50)
print("IMPORT SUMMARY")
print("="*50)
print(f"Total rows processed: {total_rows}")
print(f"Companies created: {created_count}")
print(f"Skipped (no domain): {skipped_no_domain}")
print(f"Skipped (already exists): {skipped_exists}")
print(f"Errors encountered: {error_count}")
print("="*50)

Rate limit hit for rainrescue.co.uk, sleeping for 5 seconds...

IMPORT SUMMARY
Total rows processed: 244
Companies created: 0
Skipped (no domain): 18
Skipped (already exists): 225
Errors encountered: 0
Success rate: 0.0%
